In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install PyDrive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
downloaded = drive.CreateFile({'id':"1yhOj75WuTZq7CEfeKDT4zOGMGx9QqJ0p"})   # replace the id with id of file you want to access
downloaded.GetContentFile('subset_0_of_S1B_IW_GRDH_1SDV_20200513T043115_20200513T043140_021558_028EE2_5062_TC.tif.zip')   

In [0]:
!apt install unzip

In [0]:
!unzip  subset_0_of_S1B_IW_GRDH_1SDV_20200513T043115_20200513T043140_021558_028EE2_5062_TC.tif.zip -d .

In [0]:
!apt install rasterio

In [0]:
import rasterio
image_file = "subset_0_of_S1B_IW_GRDH_1SDV_20200513T043115_20200513T043140_021558_028EE2_5062_TC.tif"
satdat = rasterio.open(image_file)



In [0]:
from osgeo import gdal
import numpy as np
from skimage import exposure
from skimage.segmentation import quickshift
from skimage.segmentation import slic


image_file = "subset_0_of_S1B_IW_GRDH_1SDV_20200513T043115_20200513T043140_021558_028EE2_5062_TC.tif"
ds = gdal.Open(image_file)
band_size = ds.RasterCount
print("total number of band",ds.RasterCount)
print(" ds share x size",ds.RasterYSize)
print(" ds share y size",ds.RasterXSize)
band_data= []
for i in range(1,band_size+1):
  band= ds.GetRasterBand(i).ReadAsArray()
  band_data.append(band.astype('float64'))

band_data = np.dstack(band_data)



# scale image values from 0.0 - 1.0
img = exposure.rescale_intensity(band_data)
 
# do segmentation multiple options with quickshift and slic
seg_start = time.time()
# segments = quickshift(img, convert2lab=False)
# segments = quickshift(img, ratio=0.8, convert2lab=False)
# segments = quickshift(img, ratio=0.99, max_dist=5, convert2lab=False)
# segments = slic(img, n_segments=100000, compactness=0.1)
# segments = slic(img, n_segments=500000, compactness=0.01)
segments = slic(img, n_segments=500000, compactness=0.1)
print('segments complete', time.time() - seg_start)

# save segments to raster
segments_fn = '/content/segment.tif'
segments_ds = driverTiff.Create(segments_fn, ds.RasterXSize, ds.RasterYSize,
                                1, gdal.GDT_Float32)
segments_ds.SetGeoTransform(ds.GetGeoTransform())
segments_ds.SetProjection(ds.GetProjectionRef())
segments_ds.GetRasterBand(1).WriteArray(segments)
segments_ds = None



